# 1. Python

### a) В python2.7 есть ф-ции range, и xrange, в чем разница?

Основной источник различий между функциями xrange и range - возвращаемые значения и тип.

range() возвращает объект типа list. Например, выражение range(1, 100, 1) создаст диапазон чисел от 0 до 99 и вернет все цифры за один раз.


С другой стороны, xrange() предоставляет результаты в виде объекта xrange. Он сохраняет аргументы и возвращает числа по запросу. В отличие от range(), он позволяет избежать получения всех чисел за один раз.

Объект xrange допускает итерацию, индексацию и метод len(). Вы можете использовать его в цикле for, чтобы обойти и получать числа в каждой итерации.

Основная проблема со стандартной функцией range() заключается в том, что она потребляет значительный объем памяти, особенно для большего диапазона чисел.

Функция xrange() всегда предоставляет следующий элемент по запросу. Это означает, что в памяти одновременно существует только один элемент. Следовательно, он потребляет меньше памяти.

### b) Написать на python 2 или 3 генератор первых n чисел фибоначи.

In [1]:
def fibonacci_generator(n):
    fib1, fib2 = 0, 1
    for _ in range(n):
        fib1, fib2 = fib2, fib1 + fib2
        yield fib1

In [2]:
n = 10
for num in fibonacci_generator(n):
              print(num, end=' ')

1 1 2 3 5 8 13 21 34 55 

# 2. Написать Sql-запросы

### 1) Вывести отдел с наибольшим числом сотрудников

SELECT d.name

FROM Department d 
  
JOIN Personal p ON d.id = p.id_dep

GROUP BY d.name

ORDER BY count(*) 

DESC LIMIT 1;

### 2) Вывести список сотрудников, получающих заработную плату выше, чем у руководителя

SELECT *

FROM Personal AS personals, Personal AS head

WHERE head.id = personals.id_head AND personals.sal > head.sal;

# 3. Задачка на определение отрасли компании

In [3]:
import pandas as pd
pays = pd.read_csv('pays.csv')
inn_info_public = pd.read_csv('inn_info_public.csv')

In [4]:
new_tab = inn_info_public[['hash_inn', 'okved2']]

In [5]:
pays_kt = pays.hash_inn_kt.value_counts()

1) Первый подход к решению задачи:

Первые две фичичи вытекают из таблицы Pays. 

Первая фича - количество платежей в среднем в неделю делает компания.

Вторая фича - средняя сумма которая компания отправляет в неделю.

Забегая вперед, провел тестирование с двумя фичами и получил точности в 28%

Следующий подход:

За фичу взять среднее количество платежей и среднюю сумму платежей для каждой отрасли.

У нас получится датасет из 162 фичей.

In [6]:
hash_inn_kt_to_week_count = pays.pivot_table(index='hash_inn_kt', # информация о каждом уникальном index
                       columns='week', # столбцы будут соответствовать уникальным columns
                       values='count', # в ячейках будет находиться информация по values
                       aggfunc='mean', # сгруппированная по aggfunc
                       fill_value=0)

### 2) Средняя сумма в неделю переводит компания

In [38]:
hash_inn_kt_to_week_sum = pays.pivot_table(index='hash_inn_kt', # информация о каждом уникальном index
                       columns='week', # столбцы будут соответствовать уникальным columns
                       values='sum', # в ячейках будет находиться информация по values
                       aggfunc='mean', # сгруппированная по aggfunc
                       fill_value=0)

In [8]:
hash_inn_kt_count_to_week = hash_inn_kt_to_week_count.mean(axis=1)\
.reset_index()\
.rename(columns={0: 'mean_count_to_week'})

In [9]:
hash_inn_kt_sum_to_week = hash_inn_kt_to_week_sum.mean(axis=1)\
.reset_index()\
.rename(columns={0: 'mean_sum_to_week'})

In [10]:
first_tab = new_tab.rename(columns={'hash_inn': 'hash_inn_kt'}).sort_values(by=['hash_inn_kt'])

In [11]:
results = first_tab.join(hash_inn_kt_count_to_week.set_index('hash_inn_kt'), on="hash_inn_kt").fillna(0)

In [12]:
results = results.join(hash_inn_kt_sum_to_week.set_index('hash_inn_kt'), on="hash_inn_kt").fillna(0)

### Сколько в среднем в неделю делает платежей кажный okved2 и средняя сумма платежей для каждого okved2

In [13]:
count_okved2 = results.pivot_table(index='hash_inn_kt', # информация о каждом уникальном index
                       columns='okved2', # столбцы будут соответствовать уникальным columns
                       values='mean_count_to_week', # в ячейках будет находиться информация по values
                       aggfunc='mean', # сгруппированная по aggfunc
                       fill_value=0).reset_index()

In [14]:
sum_okved2 = results.pivot_table(index='hash_inn_kt', # информация о каждом уникальном index
                       columns='okved2', # столбцы будут соответствовать уникальным columns
                       values='mean_sum_to_week', # в ячейках будет находиться информация по values
                       aggfunc='mean', # сгруппированная по aggfunc
                       fill_value=0).reset_index()

In [15]:
results_all = results.join(count_okved2.set_index('hash_inn_kt'), on="hash_inn_kt")

In [16]:
results_all.drop(-1, axis=1)

,hash_inn_kt,okved2,mean_count_to_week,mean_sum_to_week,0,1,2,3,4,5,...,70,71,72,73,74,75,76,77,78,79
105126,0,-1,2.383185,3148.085745,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
141899,1,62,0.583333,5.750000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
235048,2,-1,0.125000,0.874583,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
235287,3,29,0.041667,0.581167,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
111365,4,-1,0.437500,0.273472,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
142179,260511,12,0.500000,42.233333,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
181085,260512,34,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
75442,260513,12,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
34808,260514,-1,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [17]:
new_col_list = [f'count_{i}' for i in range(80)]
new_col_sum = [f'sum_{i}' for i in range(80)]

In [18]:
results_all.rename(columns={i: new_name for (i, new_name) in enumerate(new_col_list)}, inplace=True)
results_count = results_all.drop(-1, axis=1)

In [19]:
results_all = results_count.join(sum_okved2.set_index('hash_inn_kt'), on="hash_inn_kt")

In [20]:
results_count_sum = results_all.drop(-1, axis=1)

### финальный датасет для определения отрасли df_final

In [36]:
df_final = results_count_sum.query('okved2 < 0')
df_final

,hash_inn_kt,okved2,mean_count_to_week,mean_sum_to_week,count_0,count_1,count_2,count_3,count_4,count_5,...,70,71,72,73,74,75,76,77,78,79
105126,0,-1,2.383185,3148.085745,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
235048,2,-1,0.125000,0.874583,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
111365,4,-1,0.437500,0.273472,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
186982,6,-1,0.041667,0.366667,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
150864,9,-1,0.083333,1.101833,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
203346,260485,-1,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
171116,260487,-1,0.125000,635.590333,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
39729,260500,-1,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
34808,260514,-1,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### пробный датасет, который разобью на тренировочный и тестовый для определения точности модели df

In [22]:
df = results_count_sum.query('okved2 > 0')

In [23]:
# df = df.drop(['hash_inn_kt'], axis=1)

In [24]:
# df_final = df_final.drop(['hash_inn_kt'], axis=1)

### a) обучим модели

In [51]:
X = df.drop(['okved2', 'hash_inn_kt'], axis=1)
y = df['okved2']

In [52]:
from sklearn.model_selection import train_test_split

In [53]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, shuffle=True, random_state=42)
X_train

,mean_count_to_week,mean_sum_to_week,count_0,count_1,count_2,count_3,count_4,count_5,count_6,count_7,...,70,71,72,73,74,75,76,77,78,79
18229,0.250000,3.066667,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8312,1.968750,101.389167,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
218235,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
93123,1.246528,7.281083,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
58076,0.625000,36.078250,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48199,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
27401,0.083333,0.166667,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
164570,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
162104,1.458333,36.897333,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


1) DecisionTreeClassifier

In [35]:
from sklearn import tree
from sklearn.model_selection import GridSearchCV

In [36]:
clf = tree.DecisionTreeClassifier(criterion='entropy')
parameters = {'max_depth': range(1, 5), \
              'min_samples_split': range(2, 10), \
              'min_samples_leaf': range(1, 10)}

In [37]:
search = GridSearchCV(clf, parameters)

In [38]:
# search.fit(X_train, y_train)

In [39]:
y_pred = search.predict(X_test)
search.best_estimator_

In [59]:
from sklearn.metrics import precision_score, recall_score, accuracy_score
def quality_control(y_t, y_p):
    print(f"precision_score: {precision_score(y_t, y_p, average='weighted')}")
    print(f"recall_score: {recall_score(y_test, y_pred, average='weighted')}")
    print(f'accuracy_score: {accuracy_score(y_test, y_pred)}')

In [41]:
quality_control(y_test, y_pred)

### Наилучшие показатели дает XGBClassifier и RandomForestClassifier

In [54]:
from xgboost import XGBClassifier

In [55]:
xgbc = XGBClassifier()

In [56]:
xgbc.fit(X_train, y_train)

/usr/local/lib/python3.8/dist-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[12:04:15] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=4, num_parallel_tree=1,
              objective='multi:softprob', random_state=0, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=None, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

Output: XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=4, num_parallel_tree=1,
              objective='multi:softprob', random_state=0, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=None, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [57]:
y_pred = xgbc.predict(X_test)

In [60]:
quality_control(y_test, y_pred)

precision_score: 0.8599432743112646
recall_score: 0.7203856749311295
accuracy_score: 0.7203856749311295


Результаты для этой модели:

precision_score: 0.8599432743112646

recall_score: 0.7203856749311295

accuracy_score: 0.7203856749311295


In [61]:
X_test_final = df_final.drop(['okved2'], axis=1)

In [62]:
X_test1 = X_test_final.drop(['hash_inn_kt'], axis=1)

In [63]:
prediction = xgbc.predict(X_test1)
prediction

array([ 2,  1, 66, ..., 12, 12, 12])

In [64]:
new_okved = pd.DataFrame(data=prediction)
okved = new_okved.rename(columns={0: 'predict_okved'})

### Требуемый результат

Выводятся отрасти для тех компаний, в которых были значения -1

In [65]:
okved

,predict_okved
0,2
1,1
2,66
3,66
4,1
...,...
78649,12
78650,66
78651,12
78652,12


In [66]:
true_okved = okved.predict_okved

### Проверка точности модели для RandomForestClassifier

RandomForestClassifier показал такие же значения

In [70]:
from sklearn.ensemble import RandomForestClassifier

In [71]:
rf = RandomForestClassifier(n_estimators=10)

In [72]:
rf.fit(X_train, y_train)

RandomForestClassifier(n_estimators=10)

In [73]:
rf_predict = rf.predict(X_test)

In [74]:
quality_control(y_test, y_pred)

precision_score: 0.8599432743112646
recall_score: 0.7203856749311295
accuracy_score: 0.7203856749311295


## Следующим шагом, нужно взять компании получатели и получить такие же фичи

Увеличится датасет в два раза по фичам

In [84]:
true_hash_inn = new_tab.query('okved2 < 0').hash_inn
true_hash = pd.DataFrame(true_hash_inn, columns=['hash_inn'])
hash1 = true_hash.reset_index(drop=True)

In [85]:
answer = hash1.join(true_okved)

In [86]:
answer

,hash_inn,predict_okved
0,130273,2
1,205011,1
2,27440,66
3,34209,66
4,260191,1
...,...,...
78649,217719,12
78650,156954,66
78651,249271,12
78652,196702,12


In [87]:
end = answer.rename(columns={'predict_okved': 'y'})

In [89]:
end.to_csv('Vladislav-Borisov-31082021-SberbankIndustry.csv')